# Scraper para los pdfs de los convenios

In [19]:
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import pandas as pd
import asyncio
import os

In [2]:
df = pd.read_csv("data/raw/complete_scraped_data.csv")

In [3]:
adjudicados = df[df["estado"].str.contains("Adjudicado")].reset_index()

In [4]:
len(adjudicados) #just the processes that have a contract

414

In [5]:
adjudicados.head()

,index,codigo,detalle_url,entidad,objeto,estado,ubicacion,presupuesto,fecha_publicacion,semestre,...,principio_activo,forma_farmaceutica,concentracion,presentacion,cantidad,precio_referencial,subtotal,proveedor_adjudicado,valor_adjudicado,fecha_adjudicacion
0,2,SICM-500-2022,https://modulocomprascorporativas.compraspubli...,SERVICIO NACIONAL DE CONTRATACION PUBLICA,DCI: LIDOCAÍNA SIN EPINEFRINA - FORMA FARMACÉU...,Adjudicado oferente ganador,PICHINCHA / QUITO,$5.00000,2022-06-23 08:00:00,First 2022,...,Lidocaína sin epinefrina,Sólido cutáneo (Parche transdérmico),5 %,Caja x parche/parches,184631.0,USD 5.000000,"USD 923,155.000000",GRUNENTHAL ECUATORIANA CIA LTDA,USD 2.990000,2022-09-05 13:11:06
1,9,SICM-519-2022,https://modulocomprascorporativas.compraspubli...,SERVICIO NACIONAL DE CONTRATACION PUBLICA,DCI: LOPINAVIR + RITONAVIR - FORMA FARMACÉUTIC...,Adjudicado otros oferentes,PICHINCHA / QUITO,$0.27000,2022-06-23 08:00:00,First 2022,...,Lopinavir + Ritonavir,Sólido oral,200 mg + 50 mg,"Caja x frasco dosis personal, no envase hospit...",89200.0,USD 0.270000,"USD 24,084.000000",FARMAYALA PHARMACEUTICAL COMPANY S.A. FPC,USD 0.264626,2022-12-21 13:25:27
2,10,SICM-520-2022,https://modulocomprascorporativas.compraspubli...,SERVICIO NACIONAL DE CONTRATACION PUBLICA,DCI: TENOFOVIR + EMTRICITABINA - FORMA FARMACÉ...,Adjudicado otros oferentes,PICHINCHA / QUITO,$0.43854,2022-06-23 08:00:00,First 2022,...,Tenofovir + Emtricitabina,Sólido oral,300 mg + 200 mg,Caja x frasco,189144.0,USD 0.438544,"USD 82,947.966336",LABORATORIOS LACFARMA S.A.,USD 0.327000,2022-12-08 16:42:39
3,11,SICM-503-2022,https://modulocomprascorporativas.compraspubli...,SERVICIO NACIONAL DE CONTRATACION PUBLICA,DCI: MEGLUMINA ANTIMONIATO - FORMA FARMACÉUTIC...,Adjudicado oferente ganador,PICHINCHA / QUITO,$3.79707,2022-06-23 08:00:00,First 2022,...,Meglumina antimoniato,Líquido parenteral,300 mg/mL,Caja x ampolla(s) x 5 mL,18500.0,USD 3.797072,"USD 70,245.832000",OFTALVIS S.A.,USD 3.560000,2022-09-05 16:36:02
4,12,SICM-504-2022,https://modulocomprascorporativas.compraspubli...,SERVICIO NACIONAL DE CONTRATACION PUBLICA,DCI: MESALAZINA - FORMA FARMACÉUTICA: SÓLIDO O...,Adjudicado oferente ganador,PICHINCHA / QUITO,$1.30837,2022-06-23 08:00:00,First 2022,...,Mesalazina,Sólido oral (polvo),1 000 mg,Caja x sobres/sachets,199725.0,USD 1.308369,"USD 261,313.998525",NEOETHICALS CIA. LTDA.,USD 0.850000,2022-09-21 13:16:40


In [6]:
os.makedirs('convenios/', exist_ok=True)

## Build the scraper

In [7]:
url = adjudicados["detalle_url"][1]
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"

In [20]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
page = await browser.new_page(user_agent=user_agent)

url = adjudicados["detalle_url"][1]
await page.goto(url)
await asyncio.sleep(3)
await page.goto(url)

<Response url='https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/informacionProcesoContratacion2.cpe?idSoliCompra=7tz1Uy9NWcxvXO8vn5JfB5Mskdl3KZqKTK2qW5sslYI,' request=<Request url='https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/informacionProcesoContratacion2.cpe?idSoliCompra=7tz1Uy9NWcxvXO8vn5JfB5Mskdl3KZqKTK2qW5sslYI,' method='GET'>>

In [9]:
for i, row in adjudicados.iterrows():
    
    url = row["detalle_url"]
    code = row["codigo"]

    filename = f"{code}.pdf"
    save_path = f"convenios/{filename}"

    await asyncio.sleep(1)
    print(f"\rProcessing index: {i}, {url}", end="")
        
    await page.goto(url, timeout=10000)
    await asyncio.sleep(2)
            
    await page.wait_for_selector("#tab4", state="visible", timeout=10000)
    await page.eval_on_selector("#tab4", "el => el.click()")
    await asyncio.sleep(1)
    
    html = await page.content()
    soup = BeautifulSoup(html, "lxml")

    try:
        download_link_locator = page.locator(
            "//tr[translate(normalize-space(td[1]), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz')='convenio marco']//a[contains(@href, 'bajarArchivo.cpe')]"
        )
        assert await download_link_locator.count() == 1, "Expected exactly one CONVENIO MARCO link."
        
        async with page.expect_download() as download_info:
            await download_link_locator.click()

        download = await download_info.value
        await download.save_as(save_path)

        adjudicados.at[i, "contrato"] = "SI"
    
    except:
        print(f"Error at index {i}, {url}")
        adjudicados.at[i, "contrato"] = "NO"

Processing index: 36, https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/informacionProcesoContratacion2.cpe?idSoliCompra=1mz2bK48ZtXxjRoeu827fF9Sl432Dzg_fPFPxLLepB4,Error at index 36, https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/informacionProcesoContratacion2.cpe?idSoliCompra=1mz2bK48ZtXxjRoeu827fF9Sl432Dzg_fPFPxLLepB4,
Processing index: 43, https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/informacionProcesoContratacion2.cpe?idSoliCompra=lZqmJ3SKr900nPPYiYNw3RKP4duD5mUbo7k3MFI_qp4,Error at index 43, https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/informacionProcesoContratacion2.cpe?idSoliCompra=lZqmJ3SKr900nPPYiYNw3RKP4duD5mUbo7k3MFI_qp4,
Processing index: 45, https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/informacionProcesoContratacion2.cpe?idSoliCompra=5jA4rTA71S7IlEUGsERH0QtOzWVMiYHFY

In [10]:
os.makedirs('data/process', exist_ok=True)

In [11]:
adjudicados.to_csv("data/process/adjudicados_contratos.csv", index=False)

In [12]:
len(adjudicados[adjudicados["contrato"] == "NO"])

44

In [16]:
adjudicados[adjudicados["contrato"] == "NO"]["detalle_url"][36] #check one failed process

'https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/informacionProcesoContratacion2.cpe?idSoliCompra=1mz2bK48ZtXxjRoeu827fF9Sl432Dzg_fPFPxLLepB4,'

In [21]:
context = await browser.new_context()

sample_urls = adjudicados[adjudicados["contrato"] == "NO"]["detalle_url"].sample(10).tolist()

pages = []

for url in sample_urls:
    page = await context.new_page()
    await page.goto(url)
    pages.append(page)
    print(f"Opened {url}")

print(f"Opened {len(pages)} tabs. You can check them now.")

await asyncio.sleep(120)

Opened https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/informacionProcesoContratacion2.cpe?idSoliCompra=sYGKiTOfi3-2fidlUlOo7NS2Mgo1N4dUp4ykQgAgAB0,
Opened https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/informacionProcesoContratacion2.cpe?idSoliCompra=EaJoskjrKPCuPrY28o5hIqH9Jw8Q4pkzjy1yIRUHJlY,
Opened https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/informacionProcesoContratacion2.cpe?idSoliCompra=WnuJVj3pOndLJ9TQSzyZjkBss5yNF01saKGCPm19VPo,
Opened https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/informacionProcesoContratacion2.cpe?idSoliCompra=lZrxpKzL_foohBV8gJiKE17mLtqxZCIreHjMITfpaW8,
Opened https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/informacionProcesoContratacion2.cpe?idSoliCompra=_p6DurhkH1egyPGIh-7TwPsgzfK7Pm86HnTKVzwKa6Q,
Opened https://modulocomprascorporativas.compraspublica

In [22]:
#Now let's correct and loop thru the 44 with no contracts
for i, row in adjudicados[adjudicados["contrato"] == "NO"].iterrows():
    
    url = row["detalle_url"]
    code = row["codigo"]

    filename = f"{code}.pdf"
    save_path = f"convenios/{filename}"

    await asyncio.sleep(1)
    print(f"\rProcessing index: {i}, {url}", end="")
        
    await page.goto(url, timeout=10000)
    await asyncio.sleep(2)
            
    await page.wait_for_selector("#tab4", state="visible", timeout=10000)
    await page.eval_on_selector("#tab4", "el => el.click()")
    await asyncio.sleep(1)
    
    html = await page.content()
    soup = BeautifulSoup(html, "lxml")

    try:
        download_link_locator = page.locator(
            "//tr[starts-with(translate(normalize-space(td[1]), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'convenio marco')]//a[contains(@href, 'bajarArchivo.cpe')]"
        )
        assert await download_link_locator.count() == 1, "Expected exactly one CONVENIO MARCO link."
        
        async with page.expect_download() as download_info:
            await download_link_locator.click()

        download = await download_info.value
        await download.save_as(save_path)

        adjudicados.at[i, "contrato"] = "SI"
    
    except:
        print(f"Error at index {i}, {url}")
        adjudicados.at[i, "contrato"] = "NO"

Processing index: 106, https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/informacionProcesoContratacion2.cpe?idSoliCompra=4Lqpmu2fCeOpDLalJJ0-PiKmGuOjwRb1hfoPvWKW6Bo,Error at index 106, https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/informacionProcesoContratacion2.cpe?idSoliCompra=4Lqpmu2fCeOpDLalJJ0-PiKmGuOjwRb1hfoPvWKW6Bo,
Processing index: 171, https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/informacionProcesoContratacion2.cpe?idSoliCompra=XL-R5oQOu9WpKFqWAsDVUJqeMs4Z67Y4FROPjacY9Mc,Error at index 171, https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/informacionProcesoContratacion2.cpe?idSoliCompra=XL-R5oQOu9WpKFqWAsDVUJqeMs4Z67Y4FROPjacY9Mc,
Processing index: 407, https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/informacionProcesoContratacion2.cpe?idSoliCompra=2zHEVsAwAVjuz8ZJKKS7lgp7pnJ_

In [23]:
len(adjudicados[adjudicados["contrato"] == "NO"])

4

In [27]:
adjudicados.to_csv("data/process/adjudicados_contratos.csv", index=False)

In [28]:
context = await browser.new_context()

sample_urls = adjudicados[adjudicados["contrato"] == "NO"]["detalle_url"].tolist()

pages = []

for url in sample_urls:
    page = await context.new_page()
    await page.goto(url)
    pages.append(page)
    print(f"Opened {url}")

print(f"Opened {len(pages)} tabs. You can check them now.")

await asyncio.sleep(120)

Opened https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/informacionProcesoContratacion2.cpe?idSoliCompra=4Lqpmu2fCeOpDLalJJ0-PiKmGuOjwRb1hfoPvWKW6Bo,
Opened https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/informacionProcesoContratacion2.cpe?idSoliCompra=XL-R5oQOu9WpKFqWAsDVUJqeMs4Z67Y4FROPjacY9Mc,
Opened https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/informacionProcesoContratacion2.cpe?idSoliCompra=2zHEVsAwAVjuz8ZJKKS7lgp7pnJ_8JNm37jNVNnfKH4,
Opened https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/informacionProcesoContratacion2.cpe?idSoliCompra=nW3e5q1dfkXGKenYA3NULmCsTRkFXMas085_juHlkT8,
Opened 4 tabs. You can check them now.


In [30]:
playwright = await browser.close()